In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/prepped/spotify_full.csv')

In [ ]:
dftotals = df.groupby(['year', 'target'])[['track']].count()
dftotals.columns = ['n_tracks']

In [ ]:
dftotals